# トークトリアル 11 （イントロ）

# オンラインAPI/サービスを使った構造に基づくCADD

__Developed at AG Volkamer, Charité__

Dr. Jaime Rodríguez-Guerra, Dominique Sydow

## このトークトリアルの目的

Webサービスは、ユーザーのインストールの煩わしさを避けられるので、ソフトウェア利用の上で便利な方法です。Web UIは、通常ワークフローの自動化の可能性を犠牲にして、簡単に利用できるようにします。幸運にも、アクセスを自動化するためのAPI（アプリケーションプログラミングインターフェース / Application Programming Interface）を提供するWebサービスの数は増えています。コンピューター支援医薬品デザインの分野における例としては次のようなものがあります：

- ChEMBL 
- RCSB PDB
- KLIFS
- Proteins.plus
- SwissDock

以下のノートブックでは、医薬品デザインの文脈において、オンラインWebサービスをPythonからプラグラム的に使う方法を学びます。最終的な目標は、（ほとんど）すべてのローカル環境での実行なしに、もっぱらWebサービスのみに依るパイプライン全体を構築することです！

__注__: 簡単のため、このトークトリアルの実践編はイントロダクションと以下の追加の３つのノートブックに分けていてます。

- 11a. キナーゼ阻害剤の候補をKLIFとPubChemで検索
- 11b. 11aで取得した候補化合物をターゲットタンパク質に対してドッキング
- 11c. 結果を評価し既知のデータと比較

## 学習の目標

### 理論

Pythonを使ってWebサービスにプログラムでアクセスする方法の種類：

- プログラムのAPI
    - HTTPベースのRESTful API
    - どのようなAPIに対してもクライアントを生成する
- ドキュメントの構文解析（Document parsing）
- ブラウザのリモートコントロール


***


## 理論

### プログラムによるアクセスの種類

#### プログラムのAPI

現代のWebサービスは、特にデータベースの場合、データへのアクセスのために標準化された方法を提供することができます。通常、これはサイトの特定のURLにアクセスし、__機械が読むことができる形式の__結果を要求することができることを意味します。

例えば、[UniProt](https://www.uniprot.org/) はタンパク質に関するあらゆる種類の情報のためのデータベースです。もし（癌に関与している）`Src`のような特定のタンパク質を探しているのであれば、セクションごとにコンテンツがよく組織化された[この美しいWebページ](https://www.uniprot.org/uniprot/P12931)にたどり着くでしょう。ですが、もしURLに`.fast`を追加したならば、タンパク質の配列を`FASTA`形式で取得することができます。

```
https://www.uniprot.org/uniprot/P12931 -> https://www.uniprot.org/uniprot/P12931.fasta
---

>sp|P12931|SRC_HUMAN Proto-oncogene tyrosine-protein kinase Src OS=Homo sapiens OX=9606 GN=SRC PE=1 SV=3
MGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADGHRGPSAAFAPAAAE
PKLFGGFNSSDTVTSPQRAGPLAGGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGD
WWLAHSLSTGQTGYIPSNYVAPSDSIQAEEWYFGKITRRESERLLLNAENPRGTFLVRES
ETTKGAYCLSVSDFDNAKGLNVKHYKIRKLDSGGFYITSRTQFNSLQQLVAYYSKHADGL
CHRLTTVCPTSKPQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVAIKTL
KPGTMSPEAFLQEAQVMKKLRHEKLVQLYAVVSEEPIYIVTEYMSKGSLLDFLKGETGKY
LRLPQLVDMAAQIASGMAYVERMNYVHRDLRAANILVGENLVCKVADFGLARLIEDNEYT
ARQGAKFPIKWTAPEAALYGRFTIKSDVWSFGILLTELTTKGRVPYPGMVNREVLDQVER
GYRMPCPPECPESLHDLMCQCWRKEPEERPTFEYLQAFLEDYFTSTEPQYQPGENL

```

これは特定のURLスキームを介して、プログラムによりWebサービスにアクセスすることを提供する方法です。ですが、各Webサービスは各々独自の仕組みを提案しており、開発者はケースバイケースでスクリプトを実装せざるを得ません。

幸運にも、この種のプログラムによるアクセスを提供する標準化された方法がいくつかあります：

- HTTPベースのRESTful API ([wiki](https://en.wikipedia.org/wiki/Representational_state_transfer#Applied_to_Web_services))
- GraphQL
- SOAP
- gRPC

このトークトリアルでは、RESTとSOAP APIだけを扱います。

##### HTTPベースのRESTful API

この種のプログラムアクセス/規定は、プログラムによるアクセスを要求するクライアント（スクリプト、ライブラリ、プログラム）に対して特定のエントリーポイントを定義します。`api.webservice.com`といったようなものです。バージョン管理することができるので、サービス提供者はすでに存在している実装を中断せずにスキームを更新することができます（`api.webservice.com/v2` をデプロイした時でさえでも、まだ`api.webservice.com/v1`が機能し続けます）

この種のAPIには通常、プラットフォームで利用可能なアクションを全て解説したうまくまとめられたドキュメンテーションがあります。例えば、[GitHub REST API for listing repositories](https://developer.github.com/v3/repos/#list-organization-repositories)をみてください。各引数とオプションが、利用例とともにどのようにドキュメント化されているか、見てみることができるでしょう。

唯一の難しい点は、必要なURLを正しい方法で構築することです：

```
https://api.github.com/users/volkamerlab/repos
```

<details>
    <summary>
        上のURLの返り値を見るにはここをクリックしてください
    </summary>

```
[
  {
    "id": 156864288,
    "node_id": "MDEwOlJlcG9zaXRvcnkxNTY4NjQyODg=",
    "name": "TeachOpenCADD",
    "full_name": "volkamerlab/TeachOpenCADD",
    "private": false,
    "owner": {
      "login": "volkamerlab",
      "id": 44878588,
      "node_id": "MDEyOk9yZ2FuaXphdGlvbjQ0ODc4NTg4",
      "avatar_url": "https://avatars2.githubusercontent.com/u/44878588?v=4",
      "gravatar_id": "",
      "url": "https://api.github.com/users/volkamerlab",
      "html_url": "https://github.com/volkamerlab",
      "followers_url": "https://api.github.com/users/volkamerlab/followers",
      "following_url": "https://api.github.com/users/volkamerlab/following{/other_user}",
      "gists_url": "https://api.github.com/users/volkamerlab/gists{/gist_id}",
      "starred_url": "https://api.github.com/users/volkamerlab/starred{/owner}{/repo}",
      "subscriptions_url": "https://api.github.com/users/volkamerlab/subscriptions",
      "organizations_url": "https://api.github.com/users/volkamerlab/orgs",
      "repos_url": "https://api.github.com/users/volkamerlab/repos",
      "events_url": "https://api.github.com/users/volkamerlab/events{/privacy}",
      "received_events_url": "https://api.github.com/users/volkamerlab/received_events",
      "type": "Organization",
      "site_admin": false
    },
    "html_url": "https://github.com/volkamerlab/TeachOpenCADD",
    "description": "TeachOpenCADD: a teaching platform for computer-aided drug design (CADD) using open source packages and data",
    "fork": false,
    "url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD",
    "forks_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/forks",
    "keys_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/keys{/key_id}",
    "collaborators_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/collaborators{/collaborator}",
    "teams_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/teams",
    "hooks_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/hooks",
    "issue_events_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/issues/events{/number}",
    "events_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/events",
    "assignees_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/assignees{/user}",
    "branches_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/branches{/branch}",
    "tags_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/tags",
    "blobs_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/git/blobs{/sha}",
    "git_tags_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/git/tags{/sha}",
    "git_refs_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/git/refs{/sha}",
    "trees_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/git/trees{/sha}",
    "statuses_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/statuses/{sha}",
    "languages_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/languages",
    "stargazers_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/stargazers",
    "contributors_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/contributors",
    "subscribers_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/subscribers",
    "subscription_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/subscription",
    "commits_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/commits{/sha}",
    "git_commits_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/git/commits{/sha}",
    "comments_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/comments{/number}",
    "issue_comment_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/issues/comments{/number}",
    "contents_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/contents/{+path}",
    "compare_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/compare/{base}...{head}",
    "merges_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/merges",
    "archive_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/{archive_format}{/ref}",
    "downloads_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/downloads",
    "issues_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/issues{/number}",
    "pulls_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/pulls{/number}",
    "milestones_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/milestones{/number}",
    "notifications_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/notifications{?since,all,participating}",
    "labels_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/labels{/name}",
    "releases_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/releases{/id}",
    "deployments_url": "https://api.github.com/repos/volkamerlab/TeachOpenCADD/deployments",
    "created_at": "2018-11-09T13:15:15Z",
    "updated_at": "2019-07-18T02:36:48Z",
    "pushed_at": "2019-05-03T15:02:03Z",
    "git_url": "git://github.com/volkamerlab/TeachOpenCADD.git",
    "ssh_url": "git@github.com:volkamerlab/TeachOpenCADD.git",
    "clone_url": "https://github.com/volkamerlab/TeachOpenCADD.git",
    "svn_url": "https://github.com/volkamerlab/TeachOpenCADD",
    "homepage": null,
    "size": 28121,
    "stargazers_count": 39,
    "watchers_count": 39,
    "language": "Jupyter Notebook",
    "has_issues": true,
    "has_projects": true,
    "has_downloads": true,
    "has_wiki": true,
    "has_pages": false,
    "forks_count": 13,
    "mirror_url": null,
    "archived": false,
    "disabled": false,
    "open_issues_count": 0,
    "license": {
      "key": "cc-by-4.0",
      "name": "Creative Commons Attribution 4.0 International",
      "spdx_id": "CC-BY-4.0",
      "url": "https://api.github.com/licenses/cc-by-4.0",
      "node_id": "MDc6TGljZW5zZTI1"
    },
    "forks": 13,
    "open_issues": 0,
    "watchers": 39,
    "default_branch": "master"
  }
]
```
</details>

これはたまたま[JSON](https://en.wikipedia.org/wiki/JSON)フォーマットの辞書でした！`json`ライブラリを使って構文解析しPythonの辞書に変換することが簡単にできます。さらに良いことには、それをする必要すらありません。`requests`を使うことで、以下の操作が３行でできます。

In [1]:
import requests

response = requests.get("https://api.github.com/users/volkamerlab/repos")
response.raise_for_status()  # この行はエラーが生じる可能性をチェックします。
result = response.json()
result

[{'id': 222449034,
  'node_id': 'MDEwOlJlcG9zaXRvcnkyMjI0NDkwMzQ=',
  'name': 'ai_in_medicine',
  'full_name': 'volkamerlab/ai_in_medicine',
  'private': False,
  'owner': {'login': 'volkamerlab',
   'id': 44878588,
   'node_id': 'MDEyOk9yZ2FuaXphdGlvbjQ0ODc4NTg4',
   'avatar_url': 'https://avatars2.githubusercontent.com/u/44878588?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/volkamerlab',
   'html_url': 'https://github.com/volkamerlab',
   'followers_url': 'https://api.github.com/users/volkamerlab/followers',
   'following_url': 'https://api.github.com/users/volkamerlab/following{/other_user}',
   'gists_url': 'https://api.github.com/users/volkamerlab/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/volkamerlab/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/volkamerlab/subscriptions',
   'organizations_url': 'https://api.github.com/users/volkamerlab/orgs',
   'repos_url': 'https://api.github.com/users/volkamerlab/

URLを`f文字列`(`f-string`)でパラメーター化すれば、どのユーザーのレポジトリでもリストするように関数を書くことができます。

In [3]:
def repos(username):
    response = requests.get(f"https://api.github.com/users/{username}/repos")
    response.raise_for_status()
    return response.json()

repos("volkamerlab")

[{'id': 222449034,
  'node_id': 'MDEwOlJlcG9zaXRvcnkyMjI0NDkwMzQ=',
  'name': 'ai_in_medicine',
  'full_name': 'volkamerlab/ai_in_medicine',
  'private': False,
  'owner': {'login': 'volkamerlab',
   'id': 44878588,
   'node_id': 'MDEyOk9yZ2FuaXphdGlvbjQ0ODc4NTg4',
   'avatar_url': 'https://avatars2.githubusercontent.com/u/44878588?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/volkamerlab',
   'html_url': 'https://github.com/volkamerlab',
   'followers_url': 'https://api.github.com/users/volkamerlab/followers',
   'following_url': 'https://api.github.com/users/volkamerlab/following{/other_user}',
   'gists_url': 'https://api.github.com/users/volkamerlab/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/volkamerlab/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/volkamerlab/subscriptions',
   'organizations_url': 'https://api.github.com/users/volkamerlab/orgs',
   'repos_url': 'https://api.github.com/users/volkamerlab/

##### APIのクライアントを生成

便利だと思いましたか？まだ終わりではありません！

REAT APIのスキーマはプログラムにより[Swagger/OpenAPI definitions](https://swagger.io/docs/specification/about/)と呼ばれるドキュメントで表現することができます。これによりSwagger/OpenAPIスキーマを実装するREST APIのPythonクライアントを動的に生成することができます。 [これはGitHubのための実例です](https://api.apis.guru/v2/specs/github.com/v3/swagger.json)。

もちろん、Pythonで行うためのライブラリもあります。

- `bravado`
- `agithub`

Bravadoを使う場合、

In [4]:
from bravado.client import SwaggerClient
GITHUB_SWAGGER = "https://api.apis.guru/v2/specs/github.com/v3/swagger.json"
client = SwaggerClient.from_url(GITHUB_SWAGGER)
client

SwaggerClient(https://api.github.com/)

これで、メソッドとして全てのAPIアクションがあるか`client`オブジェクトを調べて遊ぶことができます。ケーススタディとして[KLIFS](http://klifs.vu-compmedchem.nl)を実例として取り上げます。

__ヒント__: clientについてこのNotebook上で調べるには`client?`を使ってください。

**訳注(2020/05)**  
`KLIFS`の実例はトークトリアル11パートAを参照してください。  
**訳注ここまで**

SOAP（Simple Object Access Protovol）のような他の標準的な方法も非常に似た原理で動作します。必要なことはクライアント生成ライブラリによって処理される定義ファイルを提供することだけです。SOAPの場合、定義ファイルは`.wsdl`(Web Services Description Language)のフォーマットとなっています。最も人気のあるライブラリの一つは`suds`です（現在、`suds-community`としてインストール可能です）。

**訳注（2020/05)**  
残念ながら現在swisdockのサーバーは機能しておらず`HTTP Error 502: Proxy Error`というエラーが表示されます。エラー表示を消すため上のセルはCodeにはしていません。　　  
**訳注ここまで**

#### ドキュメント構文解析

時折運悪く、Webサービスが、機械可読なドキュメントを生成する標準化されたAPIを提供していない場合があるでしょう。代わりに、必要な情報を得るために、通常のWebページを使って、HTMLコードを構文解析しなければなりません。これはウェブスクレイピング（(web)__scraping__）と呼ばれており、通常、
価値のあるデータを含む正しいHTMLタグとIDを見つけることを含みます（サイドバーやトップメニュー、フッター、広告 etc.といったものを無視します）。

スクレイピングでは、基本的に２つのことを行います。

1. `requests`を使ってWebページにアクセスしHTMLコンテンツを取得します。
2. `BeautifulSoup`あるいは`requests-html`で、HTML文字列を構文解析します。

この[Wikipediaの記事](https://en.wikipedia.org/wiki/Proteinogenic_amino_acid)のタンパク質を構成する(proteinogenic)アミノ酸のテーブルを構文解析して見ましょう！

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

r = requests.get("https://en.wikipedia.org/wiki/Proteinogenic_amino_acid")
r.raise_for_status()

# ここで正しいステップを推測するためには、手動でHTMLコードを調べる必要があります。
# ヒント： HTMLの定義にたどり着くにはWebページ上で右クリック＋ページの検証（inspect content）を使ってください ;)
html = BeautifulSoup(r.text)
header = html.find('span', id="General_chemical_properties")
table = header.find_all_next()[4]
table_body = table.find('tbody')

data = []
for row in table_body.find_all('tr'):
    cells = row.find_all('td')
    if cells:
        data.append([])
    for cell in cells:
        cell_content = cell.text.strip()
        try:  # 可能であればfloatに変換します
            cell_content = float(cell_content)
        except ValueError:
            pass
        data[-1].append(cell_content)
pd.DataFrame.from_records(data)

,0,1,2,3,4,5
0,A,Ala,89.09404,6.01,2.35,9.87
1,C,Cys,121.15404,5.05,1.92,10.7
2,D,Asp,133.10384,2.85,1.99,9.9
3,E,Glu,147.13074,3.15,2.1,9.47
4,F,Phe,165.19184,5.49,2.2,9.31
5,G,Gly,75.06714,6.06,2.35,9.78
6,H,His,155.15634,7.6,1.8,9.33
7,I,Ile,131.17464,6.05,2.32,9.76
8,K,Lys,146.18934,9.6,2.16,9.06
9,L,Leu,131.17464,6.01,2.33,9.74


#### ブラウザのリモートコントロールBrowser remote control

数年前のトレンドではJavaScriptを使って動的にHTMLドキュメントを生成するサーバーを構築することがあちこちで行われていました（Wikipediaのように）。言い換えれば、HTMLはサーバに組み込まれクラアント（あなたのブラウザ）に送られます。

しかし、最近のトレンドではアプリケーション全体をJavaScriptフレームワークで全部構築することに向かっています。つまり、HTMLのコンテンツがクライアント上で動的に生成されます。伝統的な構文解析は機能せず、JavaScriptフレームワークをホストする代替のHTMLコードをダウンロードするだけです。これを回避するには、HTMLはクライアント側のJavaScriptエンジンで翻訳されなければなりません。

現在のノートブックではこれは取扱いませんが、興味があれば次のプロジェクトをチェックしてください。

- [puppeteer](https://github.com/GoogleChrome/puppeteer)
- [selenium](https://www.seleniumhq.org/)

***

## ディスカッション

このイントロダクション理論編では、PythonインタープリターからオンラインのWebサービスにプログラムでアクセスする様々な方法を見てきました。これらのテクニックを活用することで、Jupyter Notebook上で自動化されたパイプラインを構築することができるようになります。このトークトリアルの続きのパートを読んでCADDへの適用例を見てください。

## クイズ

* Webサイトの特定の箇所をスクレイピングするために正確なHTMLタグとIDをみつけるためにはどうすれば良いでしょうか？自動化できるでしょうか？
* プログラムによるAPI、あるいは手動で作成したスクレイピングのどちらが良いですか？